# Tensorboard example #1: ANN

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import datetime

print('complete')


complete


In [2]:
load_ext tensorboard


In [3]:
data_path = '../remove/'

dataset = pd.read_csv(data_path + 'bank-full.csv', sep=';')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

dataset.head()


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [4]:
# binary categorical variables

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2]) #marital
X[:, 4] = le.fit_transform(X[:, 4]) #default
X[:, 6] = le.fit_transform(X[:, 6]) #housing
X[:, 7] = le.fit_transform(X[:, 7]) #loan
y = le.fit_transform(y) #target column


In [5]:
# One hot encoding of other categorical variables (> 2 values)

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1,3,8,10,15])], remainder = 'passthrough')
X = np.array(ct.fit_transform(X))


In [6]:
# train and test split

from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, test_size = 0.2, random_state = 0)


In [7]:
# scale data to unit variance

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train1 = sc.fit_transform(X_train1)
X_test1 = sc.transform(X_test1)


In [8]:
# remove existing logs, if applicable
!rm -rf ./logs/

In [9]:
# create directory to store logs
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [10]:
# define tensorboard callback for model
from tensorflow.keras.callbacks import TensorBoard 

tensorboard_callback = TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    write_graph=True,
    write_images=False,
    update_freq="epoch",
)


In [11]:
# define an ANN model
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=15, activation='relu'))
ann.add(tf.keras.layers.Dense(units=15, activation='relu'))
ann.add(tf.keras.layers.Dense(units=15, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))


In [12]:
# compile and train the model
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann.fit(X_train1, y_train1, batch_size = 32, epochs = 50, callbacks=[tensorboard_callback])


Epoch 1/50
1131/1131 [==============================] - 1s 737us/step - loss: 0.2878 - accuracy: 0.8785
Epoch 2/50
1131/1131 [==============================] - 1s 724us/step - loss: 0.2181 - accuracy: 0.9038
Epoch 3/50
1131/1131 [==============================] - 1s 679us/step - loss: 0.2110 - accuracy: 0.9054
Epoch 4/50
1131/1131 [==============================] - 1s 690us/step - loss: 0.2066 - accuracy: 0.9069
Epoch 5/50
1131/1131 [==============================] - 1s 682us/step - loss: 0.2040 - accuracy: 0.9080
Epoch 6/50
1131/1131 [==============================] - 1s 691us/step - loss: 0.2013 - accuracy: 0.9078
Epoch 7/50
1131/1131 [==============================] - 1s 691us/step - loss: 0.1993 - accuracy: 0.9089
Epoch 8/50
1131/1131 [==============================] - 1s 704us/step - loss: 0.1981 - accuracy: 0.9099
Epoch 9/50
1131/1131 [==============================] - 1s 746us/step - loss: 0.1966 - accuracy: 0.9108
Epoch 10/50
1131/1131 [==============================] - 1s 713u

In [13]:
# navigate to log folder and launch Tensorboard
!tensorboard --logdir logs/fit

# stop cell (interrupt the kernel) to quit


NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.6.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C


# Tensorboard example #2: Confusion matrix 

In [14]:
import numpy as np
import pandas as pd
import tensorflow as tf
import datetime
import sklearn

mnist = tf.keras.datasets.mnist

(X_train2, y_train2), (X_test2, y_test2) = mnist.load_data()
X_train2, X_test2 = X_train2 / 255.0, X_test2 / 255.0


In [15]:
class_names = ['Zero','One','Two','Three','Four','Five','Six','Seven','Eight','Nine']

In [16]:
import itertools
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, class_names):
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Accent)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)

    cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)
    threshold = cm.max() / 2.

    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = "white" if cm[i, j] > threshold else "black"
        plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

    return figure

In [17]:
logdir = "logs/image/"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logdir, histogram_freq = 1)
file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

In [18]:
def plot_to_image(figure):
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    plt.close(figure)
    buf.seek(0)

    digit = tf.image.decode_png(buf.getvalue(), channels=4)
    digit = tf.expand_dims(digit, 0)

    return digit

In [21]:
from tensorflow import keras
from sklearn import metrics
import io

def log_confusion_matrix(epoch, logs):
    predictions = model.predict(X_test2)
    predictions = np.argmax(predictions, axis=1)

    cm = metrics.confusion_matrix(y_test2, predictions)
    figure = plot_confusion_matrix(cm, class_names=class_names)
    cm_image = plot_to_image(figure)

    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

In [22]:
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

model = tf.keras.models.Sequential([
   tf.keras.layers.Flatten(input_shape=(28, 28)),
   tf.keras.layers.Dense(512, activation='relu'),
   tf.keras.layers.Dropout(0.2),
   tf.keras.layers.Dense(10, activation='softmax')])

model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

model.fit(X_train2, y_train2, batch_size = 32, validation_split=0.2, epochs = 30, callbacks=[tensorboard_callback, cm_callback])

Epoch 1/30
1500/1500 [==============================] - 4s 3ms/step - loss: 0.7009 - accuracy: 0.8231 - val_loss: 0.3637 - val_accuracy: 0.9039
Epoch 2/30
1500/1500 [==============================] - 4s 2ms/step - loss: 0.3687 - accuracy: 0.8970 - val_loss: 0.2961 - val_accuracy: 0.9185
Epoch 3/30
1500/1500 [==============================] - 4s 2ms/step - loss: 0.3125 - accuracy: 0.9116 - val_loss: 0.2623 - val_accuracy: 0.9268
Epoch 4/30
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2773 - accuracy: 0.9204 - val_loss: 0.2376 - val_accuracy: 0.9334
Epoch 5/30
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2527 - accuracy: 0.9303 - val_loss: 0.2185 - val_accuracy: 0.9404
Epoch 6/30
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2305 - accuracy: 0.9356 - val_loss: 0.2036 - val_accuracy: 0.9428
Epoch 7/30
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2137 - accuracy: 0.9391 - val_loss: 0.1905 - val_accuracy:

In [23]:
!tensorboard --logdir logs/image/cm/


NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.6.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C


# Tensorboard example #3: Tensorboard Projector

In [24]:
from tensorboard.plugins import projector

In [25]:
print(X_train2[0].shape)

(28, 28)


In [26]:
!rm -rf ./logs/
logdir = "logs/single-image/"
file_writer = tf.summary.create_file_writer(logdir)

In [27]:
import numpy as np

with file_writer.as_default():
    image = np.reshape(X_train2[4], (-1, 28, 28, 1))
    tf.summary.image("Single Image", image, step=0)

In [28]:
!tensorboard --logdir logs/single-image


NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.6.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C


In [29]:
!rm -rf logs
logdir = "logs/actual-images/"
file_writer = tf.summary.create_file_writer(logdir)

In [30]:
import numpy as np

with file_writer.as_default():
    images = np.reshape(X_train2[5:20], (-1, 28, 28, 1))
    tf.summary.image("Multiple Digits", images, max_outputs=16, step=0)

In [31]:
import io
import matplotlib.pyplot as plt

class_names = ['Zero','One','Two','Three','Four','Five','Six','Seven','Eight','Nine']

In [32]:
def image_grid():
    figure = plt.figure(figsize=(12,8))

    for i in range(25):
        plt.subplot(5, 5, i + 1)
        plt.xlabel(class_names[y_train[i]])
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(X_train[i], cmap=plt.cm.coolwarm)

    return figure

In [33]:
def plot_to_image(figure):
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    plt.close(figure)
    buf.seek(0)
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    image = tf.expand_dims(image, 0)
    return image

In [34]:
!tensorboard --logdir logs/actual-images


NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.6.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C


# Tensorboard example #4: Hyperparameter tuning

In [35]:
rm -rf ./logs/

In [36]:
from tensorboard.plugins.hparams import api as hp

In [37]:
logdir = "logs/hparamas"

In [38]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 17]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1,0.2))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd', 'rmsprop']))

In [39]:
METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer(logdir).as_default():
    hp.hparams_config(
        hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],)

In [40]:
def create_model(hparams):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS],  activation='relu'),
        tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
        tf.keras.layers.Dense(10, activation='softmax')])

    model.compile(optimizer=hparams[HP_OPTIMIZER],
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    model.fit(X_train2, y_train2, epochs=5)
    loss, accuracy = model.evaluate(X_test2, y_test2)

    return accuracy

In [41]:
def experiment(experiment_dir, hparams):

    with tf.summary.create_file_writer(experiment_dir).as_default():
        hp.hparams(hparams)
        accuracy = create_model(hparams)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [42]:
experiment_no = 0

for num_units in HP_NUM_UNITS.domain.values:
    for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
        for optimizer in HP_OPTIMIZER.domain.values:
            hparams = {
                HP_NUM_UNITS: num_units,
                HP_DROPOUT: dropout_rate,
                HP_OPTIMIZER: optimizer,}

            experiment_name = f'Experiment {experiment_no}'
            print(f'Starting Experiment: {experiment_name}')
            print({h.name: hparams[h] for h in hparams})
            experiment(logdir + '/' + experiment_name, hparams)
            experiment_no += 1

Starting Experiment: Experiment 0
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'adam'}
Epoch 1/5
1875/1875 [==============================] - 1s 642us/step - loss: 0.5829 - accuracy: 0.8251
Epoch 2/5
1875/1875 [==============================] - 1s 656us/step - loss: 0.3571 - accuracy: 0.8917
Epoch 3/5
1875/1875 [==============================] - 1s 644us/step - loss: 0.3168 - accuracy: 0.9042
Epoch 4/5
1875/1875 [==============================] - 1s 658us/step - loss: 0.2953 - accuracy: 0.9109
Epoch 5/5
313/313 [==============================] - 0s 457us/step - loss: 0.2027 - accuracy: 0.9395
Starting Experiment: Experiment 1
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'rmsprop'}
Epoch 1/5
1875/1875 [==============================] - 2s 703us/step - loss: 0.5603 - accuracy: 0.8323
Epoch 2/5
1875/1875 [==============================] - 1s 752us/step - loss: 0.3550 - accuracy: 0.8944
Epoch 3/5
1875/1875 [==============================] - 1s 686us/step - loss: 0.3252 - accuracy: 0.90

In [43]:
!tensorboard --logdir logs/hparamas


NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.6.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C


# Tensorboard example #5: Profiler (profile model computation performance)

In [44]:
#!pip install -U tensorboard_plugin_profile

In [45]:
device_name = tf.test.gpu_device_name()
if not device_name:
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

SystemError: GPU device not found

In [46]:
!rm -rf ./logs/
logdir = "logs"

In [47]:
from tensorflow.keras.callbacks import TensorBoard
callbacks = [TensorBoard(log_dir=logdir, profile_batch='10,20')]

In [48]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')])

model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [49]:
model.fit(X_train2, y_train2, epochs=10, validation_split=0.2, callbacks=callbacks)

Epoch 1/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.6895 - accuracy: 0.8250 - val_loss: 0.3602 - val_accuracy: 0.9043
Epoch 2/10
1500/1500 [==============================] - 4s 2ms/step - loss: 0.3684 - accuracy: 0.8978 - val_loss: 0.2939 - val_accuracy: 0.9184
Epoch 3/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3116 - accuracy: 0.9120 - val_loss: 0.2610 - val_accuracy: 0.9278
Epoch 4/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2785 - accuracy: 0.9219 - val_loss: 0.2380 - val_accuracy: 0.9327
Epoch 5/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2534 - accuracy: 0.9280 - val_loss: 0.2187 - val_accuracy: 0.9384
Epoch 6/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2340 - accuracy: 0.9334 - val_loss: 0.2042 - val_accuracy: 0.9448
Epoch 7/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2152 - accuracy: 0.9391 - val_loss: 0.1911 - val_accuracy:

In [50]:
!tensorboard --logdir logs


NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.6.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C


# Tensorboard example #6: Deep Learning Frameworks

In [57]:
from tensorboardX import SummaryWriter
import xgboost as xgb

# read in data
dtrain = xgb.DMatrix(X_train1, y_train1)
dtest = xgb.DMatrix(X_test1)

# specify parameters via map
param = {'max_depth':2, 'eta':1, 'objective':'binary:logistic' }
num_round = 2

def TensorBoardCallback():
    writer = SummaryWriter()

    def callback(env):
        for k, v in env.evaluation_result_list:
            writer.add_scalar(k, v, env.iteration)

    return callback

bst = xgb.train(param, dtrain, num_round, callbacks=[TensorBoardCallback()])

# make prediction
preds = bst.predict(dtest)
preds

array([0.06084985, 0.6253413 , 0.1690388 , ..., 0.06084985, 0.1941242 ,
       0.06084985], dtype=float32)

In [60]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter(log_dir='logs')

from torch.utils.tensorboard import SummaryWriter
import numpy as np

for n_iter in range(100):
    writer.add_scalar('Loss/train', np.random.random(), n_iter)
    writer.add_scalar('Loss/test', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/train', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/test', np.random.random(), n_iter)

In [ ]:
# run in terminal to create tensorboard.dev project
tensorboard dev upload --logdir logs \
--name "(optional) My latest experiment" \
--description "(optional) Simple comparison of several hyperparameters"

# run in terminal to revoke access
tensorboard dev auth revoke

# Tensorboard example #7: Tensorboard with Vertex AI

In [62]:
## complete section and upload to git
# https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-overview